In [2]:
%%time

import re
import os
from tqdm.notebook import tqdm
import requests
import pandas as pd
from bs4 import BeautifulSoup
from lxml import etree, html

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

chrome_driver_path = "./chromedriver-win64/chromedriver-win64/chromedriver.exe"


# 設定要爬的目錄
issue = "12-13"

df = pd.read_csv(f"csv/{issue}.csv")
download_dir  = os.path.abspath(f"data/{issue}")
os.makedirs(download_dir , exist_ok=True)

existing_files = [x.removesuffix(".pdf") for x in os.listdir(download_dir) if x != '.ipynb_checkpoints/' and x.endswith(".pdf")]
filtered_df = df[~df['name'].isin(existing_files)]
print(f"總共有{len(df)}, 剩餘{len(filtered_df)}")



# 建立 options 物件
options = Options()
options.add_argument('--headless') 
options.add_argument('--start-maximized')

# 設定下載相關偏好
options.add_experimental_option("prefs", {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})

# 建立 driver
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=options)

def wait_for_download(folder):
    seconds = 0
    while True:
        files = os.listdir(folder)
        cr_files = [f for f in files if f.endswith(".crdownload")]
        if not cr_files and files:
            return max([os.path.join(folder, f) for f in files], key=os.path.getctime)
        time.sleep(1)
        seconds += 1
        if seconds > 300:
            raise TimeoutError("下載超時")

for i, row in tqdm(filtered_df.iterrows()):
    # 開啟目標網頁
    driver.get(row['path'])
    
    # 等待網頁載入（可視需要調整）
    time.sleep(3)

    # 等待檔案下載完成
    downloaded_file_path = wait_for_download(download_dir)
    
    # 改名檔案
    new_name = os.path.join(download_dir, f"{row['name']}.pdf")
    os.rename(downloaded_file_path, new_name)
    print("下載完成：", new_name)
    
    
# 關閉瀏覽器
driver.quit()

總共有192, 剩餘192


0it [00:00, ?it/s]

下載完成： C:\Users\User\Desktop\paper_download\data\12-13\Lung Models to Evaluate Silver Nanoparticles’ Toxicity and Their Impact on Human Health.pdf
下載完成： C:\Users\User\Desktop\paper_download\data\12-13\Effect of Fabrication Method on the Thermo Mechanical and Electrical Properties of Graphene Doped PVDF Nanocomposites.pdf
下載完成： C:\Users\User\Desktop\paper_download\data\12-13\Non-Biological Slaughterhouse Wastewater Treatment with Membrane Processes—An Opportunity for Water Recycling.pdf
下載完成： C:\Users\User\Desktop\paper_download\data\12-13\Atropine-Phosphotungestate Polymeric-Based Metal Oxide Nanoparticles for Potentiometric Detection in Pharmaceutical Dosage Forms.pdf
下載完成： C:\Users\User\Desktop\paper_download\data\12-13\Optimization of Piezoresistive Strain Sensors Based on Gold Nanoparticle Deposits on PDMS Substrates for Highly Sensitive Human Pulse Sensing.pdf
下載完成： C:\Users\User\Desktop\paper_download\data\12-13\Ti3C2Tx as a Sensor for SF6_N2 Nitrogen-Containing Fault Decompositio